In [19]:
from web3 import Web3, middleware
from web3.gas_strategies.time_based import fast_gas_price_strategy
from web3.middleware import geth_poa_middleware
import json
from datetime import datetime
import pytz

# Infura節點的連接設定
infura_url = "https://data-seed-prebsc-1-s2.binance.org:8545/"

# 連接到以太坊節點
web3 = Web3(Web3.HTTPProvider(infura_url))

# Insert the POA compatibility middleware
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

web3.eth.set_gas_price_strategy(fast_gas_price_strategy)

# 要查詢的智能合約地址
contract_address = ["0xA3e241535D68Bb673635664C9B402f6766adA275", "0xD5489A9f20a4BC18198DDf7214c98b0B11561970"]
ids = [50001, 50002]
# 加載智能合約的 ABI
contract_abi = [{
    "anonymous": False,
    "inputs": [
        {
            "indexed": True,
            "internalType": "address",
            "name": "operator",
            "type": "address"
        },
        {
            "indexed": True,
            "internalType": "address",
            "name": "from",
            "type": "address"
        },
        {
            "indexed": True,
            "internalType": "address",
            "name": "to",
            "type": "address"
        },
        {
            "indexed": False,
            "internalType": "uint256[]",
            "name": "ids",
            "type": "uint256[]"
        },
        {
            "indexed": False,
            "internalType": "uint256[]",
            "name": "values",
            "type": "uint256[]"
        }
    ],
    "name": "TransferBatch",
    "type": "event"
}]

# Create a set to store unique addresses
unique_addresses = [set(), set()]

for i in range(len(contract_address)):
    contract = web3.eth.contract(address=contract_address[i], abi=contract_abi)
    gamehalf = "上半場" if i == 0 else "下半場"
    print(f"-------------------------------{gamehalf}-------------------------------")

    # 定義查詢的區塊範圍
    start_block = 29728416
    end_block = 30108462

    # 定義事件過濾器
    event_filter = contract.events.TransferBatch.build_filter()
    event_filter.fromBlock = start_block
    event_filter.toBlock = end_block
    event_filter.to = 0x0000000000000000000000000000000000000000
    event_filter.args.ids.match_single([ids[i]])

    # 獲取事件的 logs
    logs = event_filter.deploy(web3).get_all_entries()

    for event_log in logs:
        # Convert the block number to a timestamp and then to a date
        block = web3.eth.get_block(event_log['blockNumber'])
        timestamp = block['timestamp']
        date = datetime.fromtimestamp(timestamp, pytz.UTC).strftime('%Y-%m-%d %H:%M:%S')


        # Add the 'from' address to the set of unique addresses
        unique_addresses[i].add(event_log['args']['from'])

        # Print the date and the address
        print(f"{date}, {event_log['args']['from']}")

    # Print the number of unique addresses
    print(f"共 {len(unique_addresses[i])} 個 unique address")
    print("")
    print("")
    
print("")
print(f"上下半場合計共 {len(set.union(*unique_addresses))} 個 unique address")

-------------------------------上半場-------------------------------
2023-05-16 11:02:22, 0xd8eCf0C0F4e73Ee846948c4e43Bf961bfd59cC38
2023-05-16 11:14:34, 0x24A2EC7b2c28D452Ce5841128dF8433CF71Af725
2023-05-16 11:24:04, 0x07f6c0aa3c08A37F5842B691bD600BC2cCf46823
2023-05-16 11:24:10, 0x4ceB89Fe7814215914867300662c48dD0Ac2e9E4
2023-05-16 11:26:01, 0xe5106bC6E40901cb7bcD0E63b66D24D6bAEC3032
2023-05-16 11:36:34, 0xdacdF2519F24D41Af6E913E6ae06a04E51102fC2
2023-05-16 11:39:10, 0x779B6558Def8a63b0cd8e09F4bcFadB6282b7F54
2023-05-16 11:44:25, 0x8d3d6454f5dE1097d1dEbfc0d8fFeb1b16C350e1
2023-05-16 11:50:16, 0x89bEbf54C1DD2f51495d614de37845e491E9d0A7
2023-05-16 11:55:37, 0x8B6E391B73779aA9D254239B6b1375DC35402a4b
2023-05-16 11:56:31, 0x3B4E3C2c51150Ac8B0123F0223724f0077bD8885
2023-05-16 11:58:16, 0x8bFd9902cA891E4bE311cfE3EFCC45fB681622a3
2023-05-16 12:20:43, 0xc99e34CF9528E7109811D1304E458c8Bf5c0E860
2023-05-16 12:23:10, 0xa20155E761A65d7b4912dFa8D9451B38A27E799d
2023-05-16 12:26:19, 0x8dCea1Bb49Ad235

2023-05-16 23:59:55, 0x9ae9891d7d8876a496E8dF2C86e306cd48C5794e
2023-05-17 00:07:16, 0x040DE454B38E783a14ceAd7D5381D7D9b7051753
2023-05-17 00:48:55, 0x81e579bd49e2aC2BFb32aC10099E5767657E303b
2023-05-17 00:59:46, 0xdF90CC04bb26362fBE55177910EC5DD3C96090b4
2023-05-17 01:23:10, 0x888Ff81599F9f7e2A8E97dD59138CD4c3B63d3Dd
2023-05-17 01:30:49, 0x2E67e48a5921a6924940D5a2C07407653C9DCd5d
2023-05-17 01:42:49, 0x82269Dc1eEED35397B8557A38126086E2d0F3478
2023-05-17 01:52:01, 0xE4100a748E618561B4ccB97E3a660fe3ba529c10
2023-05-17 01:56:43, 0x017F16452F03C2e6A2e50E112955C8259107abd9
2023-05-17 02:47:07, 0x908EBc6b4F6D2ab6bdB1Dc7C8c2D470F07007bc1
2023-05-17 02:55:55, 0x323279Df8D205f628c2c21d16fB0550d1D61FEbD
2023-05-17 03:11:01, 0x32638078991b23Cb1a0D003F9CfD96308eC12A13
2023-05-17 03:11:52, 0x3538b1bAE0269f362373DAC7E99d144d709ba121
2023-05-17 03:23:49, 0xdc5635A17D51eb55f7fA77729482f7f744AB8982
2023-05-17 03:25:10, 0x8517F8952BB2BcF446c0E75A99d962540168b6ab
2023-05-17 03:25:28, 0xBD4A1BdC6D95540eF

2023-05-18 00:40:39, 0x1C2C59d056717712f4d2023ef5AE006c845D616f
2023-05-18 00:55:31, 0xB9EcCe94B83391d2ed4AEe237419b82F03bA777C
2023-05-18 01:01:36, 0x86dF570833fc8770f6c579639Da570612725e86f
2023-05-18 02:12:02, 0x878A1e446bdE9fF2caE9bEfF0b6549b7cf94AC83
2023-05-18 03:34:29, 0xE07fD98ed4De02ac377E3A573083382c59045ECC
2023-05-18 03:46:44, 0x30670D34127FdBd05ef63e806426188D8e39aD0D
2023-05-18 04:00:05, 0xFEd3a4a03Da60248223f78cE583C0664061ccAa0
2023-05-18 04:02:17, 0xb4A01d42Ae5F6B78DC0DA514786172c558da4BD5
2023-05-18 05:17:14, 0x6aF818CaadB63496766514570256b8933F912756
2023-05-18 06:02:08, 0x9d817F8f99C8f81d1483464442acee9F6c3cBf28
2023-05-18 06:04:29, 0x91f09148014Ea0c3173251e21B2753f80c144627
2023-05-18 06:49:20, 0xfC3Dd5C8215274150f8267a863b39B424d7D0782
2023-05-18 06:57:56, 0x56375401A4e0F3E8eeCa47eFa8EdDA95b2087389
2023-05-18 07:10:08, 0x78D2CbB9F9AB34663964E9D13b43b5aAf3329702
2023-05-18 08:10:56, 0xAE1A52C3fBd1D67139ccDA7383a2D70a4E5791bb
2023-05-18 08:43:50, 0xA8B57421A2b8ADd03

2023-05-25 10:07:35, 0xF6707F3357cDD3256149aDc23BB6d7D6CF2FC34C
2023-05-25 10:07:50, 0x4154C36b3217cD2b8Eb05607a403866d6245291D
2023-05-25 10:10:11, 0x8474738CF3ee375aA6a89Ca1fc8306E215e4fd2c
2023-05-25 10:10:59, 0x0663a31b9FA9B64e7e54Fe4d00b56Fc2881BEC34
2023-05-25 10:11:38, 0x34219BDBfDcB233FCe08185724435fDd40E89929
2023-05-25 10:11:53, 0x5F02094564a06CB09C2d6329Befa2ebF067582d5
2023-05-25 10:12:53, 0xf951C1279b86E6B64c4D25B6209a410b9C5D409C
2023-05-25 10:13:23, 0x4121Bb912370b7d3212A35fda25c57FBa1AD9eC0
2023-05-25 10:13:26, 0x8B39CCC65cFfD339c2d7A5A7215bfB55Add22206
2023-05-25 10:14:53, 0x0F37eB0D1D00927D9d1eda6f65E0B9fE737C5a5F
2023-05-25 10:15:17, 0x2Ad66777c973d51A22f282EDc52218961fBF18bf
2023-05-25 10:15:47, 0x3F709Bce4e0bBA2E68A1A3ef5D86dD30B6848598
2023-05-25 10:16:14, 0x3cF1DCd284D4201794081285ef4c4cB7969158f5
2023-05-25 10:16:32, 0x6B97d3fB8Bb0CD94348ee8D0907d9B192dc876C6
2023-05-25 10:19:53, 0x8CE93bF795d8b22076cFb5cBA51943a006c0212C
2023-05-25 10:20:05, 0x0afc2898739a0dD9A